<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Articles-Scrapper/NEWSBOMB_BIDEN_ARTICLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    # Check if the article_url_relative is already a full URL
    if article_url_relative.startswith(base_url):
        article_url = article_url_relative
    else:
        article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
biden_teasers_df = pd.read_csv("/content/drive/MyDrive/Εργασία Python/scrappers/BIDEN(21)-SCRAPER.CSV1")
biden_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,Τζο Μπάιντεν: Αναποφάσιστος ως προς τον υποχρε...,https://www.newsbomb.gr/kosmos/story/1191654/t...,2021-04-30,15:52,ΚΟΣΜΟΣ,Ο πρόεδρος Μπάιντεν δεν έχει αποφασίσει σχετικ...
1,ΗΠΑ: Οι Αμερικανοί στηρίζουν τα οικονομικά μέτ...,https://www.newsbomb.gr/kosmos/story/1191396/h...,2021-04-29,23:50,ΚΟΣΜΟΣ,Η πλειονότητα των Αμερικανών στηρίζει τα οικον...
2,ΗΠΑ: Η Αμερική «πάει μπροστά» λέει ο Μπάιντεν ...,https://www.newsbomb.gr/kosmos/story/1191143/h...,2021-04-29,08:51,ΚΟΣΜΟΣ,"Σαρωτικές προτάσεις για θέσεις εργασίας, εκπαί..."
3,ΗΠΑ: Γράφτηκε ιστορία στο Κογκρέσο - Πίσω από ...,https://www.newsbomb.gr/kosmos/story/1191106/h...,2021-04-29,06:44,ΚΟΣΜΟΣ,Κάτι που δεν έχει ξανασυμβεί διαδραματίστηκε τ...
4,Ο Κιλιντσάρογλου «σφυροκοπά» τον Ερντογάν – «Α...,https://www.newsbomb.gr/kosmos/story/1190753/o...,2021-04-28,10:19,ΚΟΣΜΟΣ,Επίθεση στον Ρετζέπ Ταγίπ Ερντογάν εξαπέλυσε η...


In [ ]:
biden_teasers_df_test = biden_teasers_df.head(10)
biden_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Τζο Μπάιντεν: Αναποφάσιστος ως προς τον υποχρε...,https://www.newsbomb.gr/kosmos/story/1191654/t...,2021-04-30,15:52,ΚΟΣΜΟΣ,Ο πρόεδρος Μπάιντεν δεν έχει αποφασίσει σχετικ...
1,ΗΠΑ: Οι Αμερικανοί στηρίζουν τα οικονομικά μέτ...,https://www.newsbomb.gr/kosmos/story/1191396/h...,2021-04-29,23:50,ΚΟΣΜΟΣ,Η πλειονότητα των Αμερικανών στηρίζει τα οικον...
2,ΗΠΑ: Η Αμερική «πάει μπροστά» λέει ο Μπάιντεν ...,https://www.newsbomb.gr/kosmos/story/1191143/h...,2021-04-29,08:51,ΚΟΣΜΟΣ,"Σαρωτικές προτάσεις για θέσεις εργασίας, εκπαί..."
3,ΗΠΑ: Γράφτηκε ιστορία στο Κογκρέσο - Πίσω από ...,https://www.newsbomb.gr/kosmos/story/1191106/h...,2021-04-29,06:44,ΚΟΣΜΟΣ,Κάτι που δεν έχει ξανασυμβεί διαδραματίστηκε τ...
4,Ο Κιλιντσάρογλου «σφυροκοπά» τον Ερντογάν – «Α...,https://www.newsbomb.gr/kosmos/story/1190753/o...,2021-04-28,10:19,ΚΟΣΜΟΣ,Επίθεση στον Ρετζέπ Ταγίπ Ερντογάν εξαπέλυσε η...
5,ΗΠΑ: Ο Τζο Μπάιντεν επισκέπτεται τον Τζίμι Κάρ...,https://www.newsbomb.gr/kosmos/story/1190699/h...,2021-04-28,08:21,ΚΟΣΜΟΣ,"Ιστορική επίσκεψη στον Τζίμι Κάρτερ, τον γηραι..."
6,Κορονοϊός: Μπάιντεν και CDC «κατεβάζουν τις μά...,https://www.newsbomb.gr/kosmos/story/1190614/k...,2021-04-27,21:59,ΚΟΣΜΟΣ,Οι πλήρως εμβολιασμένοι μπορούν να επιδίδονται...
7,Η τουρκική εθνοσυνέλευση καταδίκασε την αναγνώ...,https://www.newsbomb.gr/kosmos/story/1190567/h...,2021-04-27,19:15,ΚΟΣΜΟΣ,Η τουρκική εθνοσυνέλευση ενέκρινε ψήφισμα που ...
8,Η διπλωματία των εμβολίων: Γιατί οι ΗΠΑ ανακοί...,https://www.newsbomb.gr/kosmos/story/1190245/h...,2021-04-27,09:29,ΚΟΣΜΟΣ,Ως μια δυναμική είσοδος των ΗΠΑ στο παιχνίδι τ...
9,«Λεονταρισμοί» Ερντογάν για γενοκτονία Αρμενίω...,https://www.newsbomb.gr/kosmos/story/1190191/l...,2021-04-26,19:48,ΚΟΣΜΟΣ,Ο Τούρκος πρόεδρος Ρετζέπ Ταγίπ Ερντογάν εξέφρ...


In [ ]:
articles_df = scrape_newsbomb_articles_pages(biden_teasers_df)
articles_df

Processing rows: 100%|██████████| 564/564 [32:20<00:00,  3.44s/it]


Scraping completed. Fetshed 564 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1191654/t...,Τζο Μπάιντεν: Αναποφάσιστος ως προς τον υποχρε...,2021-04-30,15:52:22+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο πρόεδρος Μπάιντεν δεν έχει αποφασίσει σχετικ...,Ο πρόεδρος Τζο Μπάιντεν δήλωσε ότι δεν έχει ακ...,https://img.bbmd.gr/img/1260/max/82/2021/04/30...,
1,https://www.newsbomb.gr/kosmos/story/1191396/h...,ΗΠΑ: Οι Αμερικανοί στηρίζουν τα οικονομικά μέτ...,2021-04-29,23:50:49+03:00,ΚΟΣΜΟΣ,Newsbomb,Η πλειονότητα των Αμερικανών στηρίζει τα οικον...,Η δημοσκόπηση αυτή δείχνει επίσης ότι οι Ρεπου...,https://img.bbmd.gr/img/1260/max/82/2021/04/29...,
2,https://www.newsbomb.gr/kosmos/story/1191143/h...,ΗΠΑ: Η Αμερική «πάει μπροστά» λέει ο Μπάιντεν ...,2021-04-29,08:51:26+03:00,ΚΟΣΜΟΣ,Newsbomb,"Σαρωτικές προτάσεις για θέσεις εργασίας, εκπαί...",Παραμονές της συμπλήρωσης100 ημερώναπο την εκλ...,https://img.bbmd.gr/img/1260/max/82/2021/04/29...,"H πρώτη κυρία των ΗΠΑ, Τζιλ ΜπάιντενA.P"
3,https://www.newsbomb.gr/kosmos/story/1191106/h...,ΗΠΑ: Γράφτηκε ιστορία στο Κογκρέσο - Πίσω από ...,2021-04-29,06:44:41+03:00,ΚΟΣΜΟΣ,Γιάννης Φιλιππάκος,Κάτι που δεν έχει ξανασυμβεί διαδραματίστηκε τ...,"Στα 81 της, η επικεφαλής των Δημοκρατικών στο ...",https://img.bbmd.gr/img/1260/max/82/2021/04/29...,AP Photo
4,https://www.newsbomb.gr/kosmos/story/1190753/o...,Ο Κιλιντσάρογλου «σφυροκοπά» τον Ερντογάν – «Α...,2021-04-28,10:19:49+03:00,ΚΟΣΜΟΣ,Newsbomb,Επίθεση στον Ρετζέπ Ταγίπ Ερντογάν εξαπέλυσε η...,Ο ηγέτης της κεμαλικής αντιπολίτευσης (CHP)Κεμ...,https://img.bbmd.gr/img/1260/max/82/2021/04/28...,ASSOCIATED PRESS
...,...,...,...,...,...,...,...,...,...,...
559,https://www.newsbomb.gr/kosmos/story/1132775/e...,Εκλογές ΗΠΑ 2020 - Δημοσκόπηση: Προβάδισμα 7 μ...,2020-11-01,21:43:04+02:00,ΚΟΣΜΟΣ,Newsbomb,Εκλογές ΗΠΑ 2020: Στην τελική ευθεία η μάχη Τρ...,Με τις αμερικανικές προεδρικές εκλογές να απέχ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,AP
560,https://www.newsbomb.gr/kosmos/story/1132686/e...,Εκλογές ΗΠΑ 2020: Τραμπ εναντίον Μπάιντεν – Τι...,2020-11-01,14:25:49+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Προεδρικές Εκλογές ΗΠΑ 2020: Ένα 48ωρο πριν απ...,Οι ψηφοφόροι στις ΗΠΑ θα αποφασίσουν στις 3 Νο...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Τι δείχνουν οι δημοσκοπήσεις για τις εκλογές τ...
561,https://www.newsbomb.gr/kosmos/story/1132681/e...,Εκλογές ΗΠΑ 2020: Ο Τραμπ επιτίθεται στον «κοι...,2020-11-01,13:55:24+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Ο Ντόναλντ Τραμπ υποστήριξε ...,Η μάχη τωνΠροεδρικών Εκλογών στις ΗΠΑκορυφώνετ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,"Ο Αμερικανός πρόεδρος, Ντόναλντ Τραμπ\nAP Images"
562,https://www.newsbomb.gr/kosmos/story/1132645/e...,Εκλογές ΗΠΑ 2020: Κορυφώνεται η μάχη Τραμπ vs ...,2020-11-01,11:16:56+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Δυο ημέρες πριν κλείσουν οι ...,«Φωτιά» έχει πάρει το επιτελείο τουΝτόναλντ Τρ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Λιγότερες από 48 ώρες για την τελική αναμέτρησ...


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Ο πρόεδρος Τζο Μπάιντεν δήλωσε ότι δεν έχει ακόμη αποφασίσει αν όλο το προσωπικό των αντρών και γυναικών των αμερικανικών ενόπλων δυνάμεων, θα πρέπει υποχρεωτικάνα κάνουν ένα εμβόλιο κατά της COVID-19 σύμφωνα με μία συνέντευξη του, στο NBC που μεταδόθηκε σήμερα. Ο Μπάιντεν δήλωσε ότι θα είναι «μία σκληρή απόφαση» προσθέτοντας ότι μία σοβαρή σκέψη είναι το γεγονός ότι τα μέλη του στρατιωτικού προσωπικού βρίσκονται συχνά, σε μικρή απόσταση το ένα από το άλλο. Ειδήσειςαπό την Ελλάδα και τον Κόσμο, τη στιγμή που συμβαίνουν, στοNewsbomb.gr. Διαβάστε επίσης: Θρίλερ με αεροσκάφος στο Αιγαίο: Υποψία για βόμβα - Απογειώθηκαν ελληνικά μαχητικά'

In [ ]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2021/04/30/AP21119054577419.jpg?t=fGy_qELska1mSAeKwvS3qQ'

In [ ]:
from datetime import datetime

start_date = datetime(2020, 11, 20)
end_date = datetime(2021, 4, 30)

articles_df['date'] = pd.to_datetime(articles_df['date'], errors='coerce')

filtered_articles_df = articles_df[(articles_df['date'] >= start_date) & (articles_df['date'] <= end_date)].copy()

print(f"\nFiltered articles from 20/11/2020 to April 2021: {len(filtered_articles_df)}")


Filtered articles from 20/11/2020 to April 2021: 369


In [ ]:
duplicates_url = filtered_articles_df[filtered_articles_df.duplicated(subset=['article_url'], keep=False)]
print(f"\nArticles with duplicate URLs: {len(duplicates_url)}")
if not duplicates_url.empty:
    print(duplicates_url[['article_url', 'title']].sort_values(by='article_url'))

# Check for articles without title
no_title = filtered_articles_df[filtered_articles_df['title'].isnull() | (filtered_articles_df['title'].str.strip() == '')]
print(f"\nArticles without a title: {len(no_title)}")
if not no_title.empty:
    print(no_title[['article_url', 'title']])

# Check for articles without text body
no_text = filtered_articles_df[filtered_articles_df['text_body'].isnull() | (filtered_articles_df['text_body'].str.strip() == '')]
print(f"\nArticles without text body: {len(no_text)}")
if not no_text.empty:
    print(no_text[['article_url', 'title']])


Articles with duplicate URLs: 0

Articles without a title: 0

Articles without text body: 9
                                           article_url  \
45   https://www.newsbomb.gr/kosmos/video/1188315/m...   
69   https://www.newsbomb.gr/kosmos/video/1183939/t...   
86   https://www.newsbomb.gr/kosmos/video/1181183/t...   
129  https://www.newsbomb.gr/kosmos/video/1178476/m...   
138  https://www.newsbomb.gr/kosmos/video/1177453/n...   
182  https://www.newsbomb.gr/kosmos/video/1164362/h...   
217  https://www.newsbomb.gr/kosmos/video/1159378/o...   
254  https://www.newsbomb.gr/kosmos/video/1158986/o...   
341  https://www.newsbomb.gr/kosmos/video/1142715/t...   

                                                 title  
45   Μπάιντεν για καταδίκη Σόβιν: Mπορεί να αποτελέ...  
69   Τέξας: Η σπαρακτική κραυγή ενός παιδιού μετανα...  
86   Tραμπ σε γαμήλιο πάρτι στη Φλόριντα: «Σας έλει...  
129  Μπάιντεν: Μπέρδεψε τα πόδια του! Έπεσε τρεις φ...  
138  Ντοναλντ Τραμπ: Συνιστώ να κάνετε το

In [ ]:
biden_articles_21 = filtered_articles_df

In [ ]:
biden_articles_21 = biden_articles_21.drop([45, 69, 86, 129, 138, 182, 217, 254, 341])

In [ ]:
from google.colab import drive

In [ ]:
# Assuming you want to scrape the articles from the filtered_articles_df
# If you intended to scrape from the original biden_teasers_df, replace filtered_articles_df with biden_teasers_df
articles_df_scraped = scrape_newsbomb_articles_pages(filtered_articles_df)

# Save the scraped articles DataFrame to CSV
articles_df_scraped.to_csv('/content/drive/MyDrive/Εργασία Python/Articles/biden_articles_20_21.csv', index=False)

Processing rows: 100%|██████████| 369/369 [20:50<00:00,  3.39s/it]


Scraping completed. Fetshed 369 articles.
